In [10]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.preprocessing import FunctionTransformer, KBinsDiscretizer, StandardScaler, MinMaxScaler, PolynomialFeatures, RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.pipeline import make_pipeline
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [11]:
data_path = "Dataset_Brief.csv"
df = pd.read_csv(data_path)

In [12]:
duplicates = df[df.duplicated(keep=False)]
print(duplicates)
index_to_remove = 581
df = df.drop(index_to_remove)
df = df.reset_index(drop=True)
print(df.shape)

     age   sex    bmi  children smoker     region    charges
195   19  male  30.59         0     no  northwest  1639.5631
581   19  male  30.59         0     no  northwest  1639.5631
(1337, 7)


In [13]:


categories = {
    'Underweight': (0, 18.5),
    'Normal Weight': (18.5, 24.9),
    'Overweight': (25, 29.9),
    'Obesity Class I': (30, 34.9),
    'Obesity Class II': (35, 39.9),
    'Obesity Class III': (40, float('inf'))
}

# Fonction pour attribuer la catégorie en fonction de la valeur d'IMC
def categorize_imc(bmi):
    for category, (lower, upper) in categories.items():
        if lower <= bmi <= upper:
            return category
        

df['imc_category'] = df['bmi'].apply(categorize_imc)


In [14]:

df_modelisation = df.copy()
df_modelisation = df_modelisation.drop('bmi', axis=1)


In [15]:
df_modelisation.to_csv('Dataset_modelisation.csv', index=False)
df_modelisation

,age,sex,children,smoker,region,charges,imc_category
0,19,female,0,yes,southwest,16884.92400,Overweight
1,18,male,1,no,southeast,1725.55230,Obesity Class I
2,28,male,3,no,southeast,4449.46200,Obesity Class I
3,33,male,0,no,northwest,21984.47061,Normal Weight
4,32,male,0,no,northwest,3866.85520,Overweight
...,...,...,...,...,...,...,...
1332,50,male,3,no,northwest,10600.54830,Obesity Class I
1333,18,female,0,no,northeast,2205.98080,Obesity Class I
1334,18,female,0,no,southeast,1629.83350,Obesity Class II
1335,21,female,0,no,southwest,2007.94500,Overweight


In [16]:
df_analyse = df.copy()

df_analyse['smoker'] = df_analyse['smoker'].replace({'yes': 1, 'no': 0})
df_analyse['sex'] = df_analyse['sex'].replace({'female': 1, 'male': 0})

df_analyse['bmi'] = df_analyse['bmi'].astype(float)
df_analyse['age'] = df_analyse['age'].astype(int)
df_analyse['children'] = df_analyse['children'].astype(int)

In [17]:
object_encodage = ['region','imc_category']

for col in object_encodage:
    unique_values = df_analyse[col].unique()
    for value in unique_values:
        df_analyse[f"{col}_{value}"] = (df_analyse[col] == value).astype(int)

df_analyse = df_analyse.drop(object_encodage, axis=1)

In [18]:
df_analyse.to_csv('Dataset_analyse.csv', index=False)
df_analyse

,age,sex,bmi,children,smoker,charges,region_southwest,region_southeast,region_northwest,region_northeast,imc_category_Overweight,imc_category_Obesity Class I,imc_category_Normal Weight,imc_category_Obesity Class II,imc_category_Obesity Class III,imc_category_Underweight,imc_category_None
0,19,1,27.900,0,1,16884.92400,1,0,0,0,1,0,0,0,0,0,0
1,18,0,33.770,1,0,1725.55230,0,1,0,0,0,1,0,0,0,0,0
2,28,0,33.000,3,0,4449.46200,0,1,0,0,0,1,0,0,0,0,0
3,33,0,22.705,0,0,21984.47061,0,0,1,0,0,0,1,0,0,0,0
4,32,0,28.880,0,0,3866.85520,0,0,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,50,0,30.970,3,0,10600.54830,0,0,1,0,0,1,0,0,0,0,0
1333,18,1,31.920,0,0,2205.98080,0,0,0,1,0,1,0,0,0,0,0
1334,18,1,36.850,0,0,1629.83350,0,1,0,0,0,0,0,1,0,0,0
1335,21,1,25.800,0,0,2007.94500,1,0,0,0,1,0,0,0,0,0,0
